<a href="https://colab.research.google.com/github/kwanglo/mge51101-20195171/blob/master/final_project/utterance_demo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!pip3 install konlpy
!pip3 install soynlp

In [0]:
import os
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Twitter

import pandas as pd
import numpy as np

In [0]:
path='/gdrive/My Drive/Colab Notebooks/Final Project/dataset/'
df = pd.read_csv(path+"fci_data.csv")
#train_set = pd.read_csv(path+"fci_train.csv")
#test_set = pd.read_csv(path+"fci_test.csv")

In [6]:
df.groupby("label").count()
#Fragments(FR) - 0
#Statements(S) - 1
#Questions(Q) - 2
#Commands(C) - 3
#Rhetorical questions(RQ) - 4
#Rhetorical commands(RC) - 5
#FragmeIntonation-dependent utterances(IU) - 6

,text
label,
0,6009
1,18300
2,17869
3,12968
4,1745
5,1087
6,3277


# Preprocessing data

In [0]:
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *
import re
from konlpy.tag import Okt

def tokenizer(text): # create a tokenizer function
    okt = Okt()
    text = only_hangle(text)
    text = repeat_normalize(text, num_repeats = 2)
    x = okt.morphs(text , stem= True)
    return x

In [0]:
stop_words_set = pd.read_csv(path+'stopwords100.txt',header = 0, delimiter = '\t', quoting = 3)
stop_words= (list(stop_words_set['aa']))
stop_words2 = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words.extend(stop_words)

In [0]:
import torch
from torchtext import data
from torchtext import datasets
from soynlp.tokenizer import MaxScoreTokenizer
SEED = 3432

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = data.Field(tokenize=tokenizer, stop_words = stop_words)
LABEL = data.LabelField()

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [0]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(df['text'],
                                                                        df['label'], 
                                                                        random_state=2020, 
                                                                        test_size=0.3)

In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_inputs,
                                                                                    train_labels, 
                                                                                    random_state=2020, 
                                                                                    test_size=0.3)

In [0]:
train_set = pd.concat([train_inputs, train_labels], axis=1)
valid_set = pd.concat([validation_inputs, validation_labels], axis=1)
test_set = pd.concat([test_inputs, test_labels], axis=1)

In [0]:
train_set.to_csv(path+'utterance_train.csv',index=False, encoding='utf-8')
valid_set.to_csv(path+'utterance_valid.csv',index=False, encoding='utf-8')
test_set.to_csv(path+'utterance_test.csv',index=False, encoding='utf-8')

In [0]:
from torchtext.data import TabularDataset
fields = [("text", TEXT),("label", LABEL)]

In [0]:
train_data,valid_data, test_data = data.TabularDataset.splits(
                                        path = path,
                                        train = 'utterance_train.csv',
                                        validation = 'utterance_valid.csv',
                                        test = 'utterance_test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [0]:
import torchtext
vec = torchtext.vocab.Vectors('wiki.ko.vec', cache=path)

In [0]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = vec, 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [40]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f26b7591400>, {'1': 0, '2': 1, '3': 2, '0': 3, '6': 4, '4': 5, '5': 6})


In [0]:
from torchtext.data import Iterator, BucketIterator
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = BucketIterator.splits(
 (train_data, valid_data, ), 
 batch_sizes=(BATCH_SIZE, BATCH_SIZE),
 device=device,
 sort_key=lambda x: len(x.Sentence), 
 sort_within_batch=False, sort=False,
 repeat=False
)

test_iterator  = Iterator(test_data, batch_size=BATCH_SIZE, device= device, sort_key=lambda x: len(x.Sentence), sort_within_batch=False, sort=False, repeat=False)

In [0]:
from torchtext.data import Iterator, BucketIterator
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort = False)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        text = text.permute(1, 0)        
        embedded = self.embedding(text)

        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [45]:
len(TEXT.vocab)

9821

In [46]:
len(LABEL.vocab)

7

In [47]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,218,707 trainable parameters


In [48]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-8.9922e-02, -5.2143e-02, -4.3379e-01,  ...,  5.2252e-01,
         -9.1893e-02,  3.7348e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-2.1333e-01, -1.4143e-03, -9.6966e-01,  ..., -3.1586e-01,
          1.9719e-01,  5.1325e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [58]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 59s
	Train Loss: 1.056 | Train Acc: 61.09%
	 Val. Loss: 0.925 |  Val. Acc: 66.69%
Epoch: 02 | Epoch Time: 1m 3s
	Train Loss: 0.819 | Train Acc: 70.21%
	 Val. Loss: 0.911 |  Val. Acc: 68.14%
Epoch: 03 | Epoch Time: 1m 3s
	Train Loss: 0.670 | Train Acc: 75.23%
	 Val. Loss: 0.963 |  Val. Acc: 67.23%
Epoch: 04 | Epoch Time: 1m 3s
	Train Loss: 0.553 | Train Acc: 79.32%
	 Val. Loss: 1.036 |  Val. Acc: 66.47%
Epoch: 05 | Epoch Time: 1m 4s
	Train Loss: 0.469 | Train Acc: 82.04%
	 Val. Loss: 1.154 |  Val. Acc: 66.41%
Epoch: 06 | Epoch Time: 1m 4s
	Train Loss: 0.402 | Train Acc: 84.55%
	 Val. Loss: 1.271 |  Val. Acc: 66.03%
Epoch: 07 | Epoch Time: 1m 5s
	Train Loss: 0.357 | Train Acc: 85.98%
	 Val. Loss: 1.363 |  Val. Acc: 66.04%
Epoch: 08 | Epoch Time: 1m 5s
	Train Loss: 0.324 | Train Acc: 87.24%
	 Val. Loss: 1.499 |  Val. Acc: 65.82%
Epoch: 09 | Epoch Time: 1m 4s
	Train Loss: 0.295 | Train Acc: 88.29%
	 Val. Loss: 1.664 |  Val. Acc: 66.35%
Epoch: 10 | Epoch Time: 1m 

In [59]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.914 | Test Acc: 67.96%


In [60]:
tokenizer("오늘 되게 우울하다")

['오늘', '되게', '우울하다']

In [0]:
def predict_class(model, sentence, min_len = 4):
    model.eval()
    # 이 부분에서 그냥 바로 tokenizing
    tokenized = tokenizer(sentence)
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [0]:
def utterance_classifier(logits):
  global utterance
  if logits == 0:
    utterance = '미완'
  elif logits == 1:
    utterance = '서술'
  elif logits == 2:
    utterance = '질문'
  elif logits == 3:
    utterance = '요구'
  elif logits == 4:
    utterance = '수사의문'
  elif logits == 5:
    utterance = '수사명령'
  elif logits == 6:
    utterance = '억양'

  return utterance

In [83]:
pred_sentence = input()
pred_class = predict_class(model, pred_sentence)
logit = LABEL.vocab.itos[pred_class]

print(f'이 문장의 발화 의도는: {utterance_classifier(logit)}')

내일부터 일주일간 뭐해?
이 문장의 발화 의도는: 서술


In [81]:
temp = df[df["label"]==2]
temp

,label,text
21879,2,내일 날씨 맑니
21880,2,세탁기 전원 꺼졌
21881,2,내일 오후 일정
21882,2,오늘 점심때 마사
21883,2,이번달 결혼기념일
...,...,...
59345,2,창문 다 닫혀있니
59346,2,옷방에 불 꺼져
59347,2,전자시계가 배터리
59348,2,내게 쓴 메일함에
